# Relatório Banco de Dados
Alex Moulin, Pedro Ribeiro

In [4]:
pip install pandasql

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26768 sha256=daf3c7088e29861b0436c48168283b13b6c95d0f3233e78e63dd434624e33055
  Stored in directory: c:\users\antph\appdata\local\pip\cache\wheels\63\e8\ec\75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandasql as ps 
import pandas as pd

In [2]:
df = pd.read_csv("vgsales.csv")
df.shape

(16598, 11)

In [3]:
df = df.dropna()

In [4]:
df.isnull().sum()

Rank            0
Name            0
Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64

In [5]:
pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd 
import psycopg2

Criando Schema e Tabelas

In [23]:
conn = psycopg2.connect(
    host = "localhost", 
    database = "postgres",  
    user = "postgres", 
    password = "126474"
)
cur = conn.cursor()

comando = '''
DROP SCHEMA games CASCADE;
CREATE SCHEMA games;
'''
cur.execute(comando)
conn.commit()
cur.close()
conn.close()

In [24]:
conn = psycopg2.connect(
    host = "localhost", 
    database = "postgres",  
    user = "postgres", 
    password = "126474"
)
cur = conn.cursor()

comando = '''

CREATE TABLE games.plataforma (
  id_plataforma SERIAL PRIMARY KEY,
  plataforma varchar(40)
);

CREATE TABLE games.genero (
  id_genero SERIAL PRIMARY KEY,
  genero varchar(40)
);

CREATE TABLE games.publisher (
  id_publisher SERIAL PRIMARY KEY,
  publisher varchar(40)
);

CREATE TABLE games.jogos (
  id_jogo SERIAL PRIMARY KEY,
  jogo varchar(200),
  ano integer,
  cod_plataforma integer,
  cod_genero integer,
  cod_publisher integer,
  FOREIGN KEY (cod_plataforma) REFERENCES games.plataforma (id_plataforma),
  FOREIGN KEY (cod_genero) REFERENCES games.genero (id_genero),
  FOREIGN KEY (cod_publisher) REFERENCES games.publisher (id_publisher)
);


CREATE TABLE games.vendas (
    cod_jogo INTEGER,
    na float,
    eu float,
    jp float,
    ot float,
    global float,
    FOREIGN KEY (cod_jogo) REFERENCES games.jogos (id_jogo)
);
'''
cur.execute(comando)
conn.commit()
cur.close()
conn.close()

Tratando nomes com apostrofo que aparecem no banco de dados, Black's -> Blacks

In [12]:
for i in range(df.shape[0]):
    df.Name.values[i] = df.Name.values[i].replace("'","")
    df.Publisher.values[i] = df.Publisher.values[i].replace("'","")

In [7]:
import warnings
warnings.filterwarnings('ignore')

Cria dicionários uteis para popular a base de dados.

In [25]:
dici_plat = dict()
k=1
for i in df.Platform.unique():
    dici_plat.update({i:k})
    k+=1

dici_genero = dict()
k=1
for i in df.Genre.unique():
    dici_genero.update({i:k})
    k+=1
    
dici_pub = dict()
k=1
for i in df.Publisher.unique():
    dici_pub.update({i:k})
    k+=1
    


Preenchendo a base de dados

In [ ]:
conn = psycopg2.connect(
    host = "localhost", 
    database = "postgres",  
    user = "postgres", 
    password = "126474"
)

for i in dici_plat:
    cur = conn.cursor()
    query=f"""INSERT INTO games.plataforma (plataforma) VALUES ('{i}')"""
    cur.execute(query)
    cur.close()
    
for i in dici_genero:
    cur = conn.cursor()
    query=f"""INSERT INTO games.genero (genero) VALUES ('{i}')"""
    cur.execute(query)
    cur.close()
    
for i in dici_pub:
    cur = conn.cursor()
    query=f"""INSERT INTO games.publisher (publisher) VALUES ('{i}')"""
    cur.execute(query)
    cur.close()



for i in range(df.shape[0]):
    
    query4=f"""INSERT INTO games.jogos (jogo,ano,cod_plataforma,cod_genero,cod_publisher) VALUES ('{df.iloc[:,1].values[i]}',{int(df.iloc[:,3].values[i])},{dici_plat[df.Platform.values[i]]},{dici_genero[df.Genre.values[i]]},{dici_pub[df.Publisher.values[i]]}) RETURNING @id_jogo; 
        SELECT LASTVAL() AS var;
        """
    cod = pd.read_sql_query(query4, con = conn)
    c_jogo = cod['var'][0]
    
    cur = conn.cursor()
    query5=f"""INSERT INTO games.vendas (cod_jogo,na,eu,jp,ot,global) VALUES ({c_jogo},{df.iloc[:,6].values[i]},{df.iloc[:,7].values[i]},{df.iloc[:,8].values[i]},{df.iloc[:,9].values[i]},{df.iloc[:,10].values[i]});
    COMMIT"""
    
    cur.execute(query5)
    cur.close()
conn.close()

# Perguntas!!!

In [5]:
conn = psycopg2.connect(
    host = "localhost", 
    database = "postgres",  
    user = "postgres", 
    password = "126474"
)

1) Quais são os 10 jogos mais comprados em TODAS as plataformas em todos os anos

In [10]:
com = """
SELECT * FROM (
	SELECT j.jogo, SUM(v.global) as global
	FROM games.jogos j 
	INNER JOIN games.vendas v ON j.id_jogo = v.cod_jogo
	GROUP BY j.jogo
	) t
ORDER BY t.global DESC
LIMIT 10;
"""
cod = pd.read_sql_query(com, con = conn)
cod

,jogo,global
0,Wii Sports,82.74
1,Grand Theft Auto V,55.92
2,Super Mario Bros.,45.31
3,Tetris,35.84
4,Mario Kart Wii,35.82
5,Wii Sports Resort,33.00
6,Pokemon Red/Pokemon Blue,31.37
7,Call of Duty: Modern Warfare 3,30.83
8,New Super Mario Bros.,30.01
9,Call of Duty: Black Ops II,29.72


 2) Qual a plataforma que vendeu mais no Japão?

In [11]:
com = """
SELECT * FROM (
	SELECT p.plataforma, SUM(v.jp) as jp FROM games.plataforma p
	INNER JOIN games.jogos j ON j.cod_plataforma = p.id_plataforma
	INNER JOIN games.vendas v ON j.id_jogo = v.cod_jogo
	GROUP BY p.plataforma
	) t
ORDER BY t.jp DESC
LIMIT 10;
"""
cod = pd.read_sql_query(com, con = conn)
cod

,plataforma,jp
0,DS,175.02
1,PS,139.78
2,PS2,137.54
3,SNES,116.55
4,NES,98.65
5,3DS,97.30
6,GB,85.12
7,PS3,79.21
8,PSP,75.89
9,Wii,68.28


3) Qual o ano em que mais jogos foram vendidos?

In [12]:
com = """
SELECT * FROM (
	SELECT j.ano, SUM(v.global) as global
	FROM games.jogos j 
	INNER JOIN games.vendas v ON j.id_jogo = v.cod_jogo
	GROUP BY j.ano
	) t
ORDER BY t.global DESC
LIMIT 10;
"""
cod = pd.read_sql_query(com, con = conn)
cod

,ano,global
0,2008,678.90
1,2009,667.30
2,2007,609.92
3,2010,600.29
4,2006,521.04
5,2011,515.80
6,2005,458.51
7,2004,414.01
8,2002,395.52
9,2013,368.11


4) Qual publisher fez mais jogos de estratégia entre 2000 e 2010?

In [13]:
com = """
SELECT * FROM (
	SELECT p.publisher, COUNT(g.genero) as genero
	FROM games.publisher p
	INNER JOIN games.jogos j ON p.id_publisher = j.cod_publisher
	INNER JOIN games.genero g ON j.cod_genero = g.id_genero
	WHERE g.genero = 'Strategy' and j.ano BETWEEN 2000 AND 2010
	GROUP BY p.publisher
	) t
ORDER BY t.genero DESC
LIMIT 10;
"""
cod = pd.read_sql_query(com, con = conn)
cod

,publisher,genero
0,Namco Bandai Games,34
1,THQ,30
2,Electronic Arts,26
3,Ubisoft,24
4,Tecmo Koei,23
5,Nintendo,21
6,Sega,21
7,Konami Digital Entertainment,21
8,Atari,17
9,Banpresto,17


5) Qual publisher tem jogos em mais plataformas?

In [14]:
com = """
SELECT * FROM (
	SELECT p.publisher, COUNT(DISTINCT k.plataforma) as plataforma
	FROM games.publisher p
	INNER JOIN games.jogos j ON p.id_publisher = j.cod_publisher
	INNER JOIN games.plataforma k ON j.cod_plataforma = k.id_plataforma
	GROUP BY p.publisher
	) t
ORDER BY t.plataforma DESC
LIMIT 10;
"""
cod = pd.read_sql_query(com, con = conn)
cod

,publisher,plataforma
0,Namco Bandai Games,23
1,Konami Digital Entertainment,23
2,Capcom,23
3,Sega,21
4,Activision,20
5,Ubisoft,19
6,Electronic Arts,19
7,Hudson Soft,17
8,Take-Two Interactive,17
9,Tecmo Koei,17


6) Qual a média de vendas globais de jogos de cada genero?

In [15]:
com = """
SELECT * FROM (
	SELECT g.genero, AVG(v.global) as global
	FROM games.genero g
	INNER JOIN games.jogos j ON g.id_genero = j.cod_genero
	INNER JOIN games.vendas v ON j.id_jogo = v.cod_jogo
	GROUP BY g.genero
	) t
ORDER BY t.global DESC
"""
cod = pd.read_sql_query(com, con = conn)
cod

,genero,global
0,Platform,0.947577
1,Shooter,0.800468
2,Role-Playing,0.628456
3,Racing,0.593273
4,Sports,0.568247
5,Fighting,0.531160
6,Action,0.529942
7,Misc,0.468488
8,Simulation,0.459882
9,Puzzle,0.424930


7) Quais os anos de lançamentos de jogos da franquia Elder Scrolls?

In [16]:
com = """
SELECT DISTINCT j.jogo, j.ano AS ano 
FROM games.jogos as j
WHERE j.jogo LIKE 'The Elder Scrolls%'
ORDER BY j.ano
"""
cod = pd.read_sql_query(com, con = conn)
cod

,jogo,ano
0,The Elder Scrolls III: Morrowind,2002
1,The Elder Scrolls IV: Oblivion,2006
2,The Elder Scrolls IV: Oblivion,2007
3,The Elder Scrolls V: Skyrim,2011
4,The Elder Scrolls Online,2014
5,The Elder Scrolls Online,2015


8) Qual mercado a Nintendo vendeu mais jogos de Aventura?

In [21]:
com = """
SELECT SUM(v.na) as America_Norte,SUM(v.eu) as Europa ,SUM(v.jp)as Japao ,SUM(v.ot) as Outros
FROM games.vendas v
INNER JOIN games.jogos j ON j.id_jogo = v.cod_jogo
INNER JOIN games.publisher p ON p.id_publisher = j.cod_publisher
INNER JOIN games.genero g ON j.cod_genero = g.id_genero
WHERE g.genero = 'Adventure' AND p.publisher = 'Nintendo'
"""
cod = pd.read_sql_query(com, con = conn)
cod

,america_norte,europa,japao,outros
0,17.72,7.66,9.01,1.28


9) Qual o genero mais jogado no ano de 2013?

In [23]:
com = """
SELECT * FROM (
	SELECT g.genero, SUM(v.global) as global
	FROM games.genero g
	INNER JOIN games.jogos j ON j.cod_genero = g.id_genero
	INNER JOIN games.vendas v ON j.id_jogo = v.cod_jogo
	WHERE j.ano=2013
	GROUP BY g.genero
	
) t
ORDER BY t.global DESC
LIMIT 10;
"""
cod = pd.read_sql_query(com, con = conn)
cod

,genero,global
0,Action,125.22
1,Shooter,62.80
2,Role-Playing,44.92
3,Sports,41.55
4,Misc,25.65
5,Platform,25.12
6,Racing,13.04
7,Simulation,8.67
8,Fighting,7.21
9,Adventure,6.61


10) Qual a porcentagem de vendas na Europa em relação às vendas Globais até 1995?

In [24]:
com = """
SELECT SUM(v.eu)/SUM(v.global)*100 as percentagem
FROM games.vendas v
INNER JOIN games.jogos j ON j.id_jogo = v.cod_jogo
WHERE j.ano < 1995
"""
cod = pd.read_sql_query(com, con = conn)
cod

,percentagem
0,11.223484
